In [1]:
# !pip install skimpy --quiet
# !pip install wordcloud --quiet
# !pip install category_encoders --quiet
# !pip install shap
# pip install colorama
from colorama import Fore, Style
# Basic imports
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd
#import skimpy
import re
import time
import random
import datetime
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", message="Parameters: { 'verbose' } are not used.")

##################### Preprocessing imports 
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import mutual_info_classif, mutual_info_regression
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from category_encoders import TargetEncoder
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedStratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingRegressor
from sklearn.inspection import permutation_importance
from sklearn.base import clone
from sklearn.metrics import cohen_kappa_score
from scipy.optimize import minimize
from IPython.display import clear_output

##################### Models
from xgboost import XGBClassifier, XGBRegressor
from catboost import CatBoostClassifier, CatBoostRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline

##################### optuna library import
import optuna
import shap
random_state = 42
n_splits = 5

#!pip install opendatasetS

c:\Users\Nadav\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [ ]:
print("hello")

In [2]:
######## Download the data 
train= pd.read_csv("../data/raw/train.csv")
test=pd.read_csv("../data/raw/test.csv")
sample = pd.read_csv('../data/raw/sample_submission.csv')
data_dict=pd.read_csv("../data/raw/data_dictionary.csv")

<h3> Basic configuration </h3>
<li>Map sii - map the target column in order to make the target data more clear</li>
<li>perform basic actions - such as dropping the id column across the datasets and applying the map for the dataset</li>

In [ ]:
try:
    # Define Dependent and Target feature 
    target = 'sii'
    Target_series = train['sii']
    
    # Drop the id column from all dataset respectively 
    train.drop(['id'], axis =1, inplace = True)
    test.drop(['id'], axis =1, inplace = True)
    data_dict = data_dict[data_dict['Field'].str.contains('id') == False]
    
    # Create a copy 
    train_copy = train.copy()
 
except:
    print("Already dropped id or name column")
    

<h1>Preprocessing action</h1>
<p>In this section I am creating all the necassries function for the feature engineering part of the tabular dataset </p>

In [ ]:
def concat_df_a_target(df,target):
    return pd.concat([df,target], axis=1)

def drop_Nans(train,subset):
    dropped_df = train.dropna(subset=subset).reset_index().drop('index',axis=1)
    return dropped_df

def feature_difference(train, test):
    # Get the set of column names from each DataFrame
    train_set = set(train.columns)
    test_set = set(test.columns)

    # find the difference in cols
    feature_difference_cols = train_set - test_set

    return feature_difference_cols


def cap_outliers(train, columns, method='iqr', threshold=1.5):
    '''
    for the this tests I only want to remove outliers for some of the features and not all.
    For example if I am not cartefull the function will alter the target column 
    because of that it is important to set to change only the desired columns
    '''
    
    train_copy = train.copy()
    
    for col in columns:
        if col != 'sii':
            Q1 = train[col].quantile(0.25)
            Q3 = train[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - threshold * IQR
            upper_bound = Q3 + threshold * IQR

            train_copy[col] = np.clip(train[col], lower_bound, upper_bound)
        
    return train_copy


# This function is not neccasry
def correct_outliers_dk(df):
    train = df.copy()
    # Define thresholds
    bmi_threshold = 7
    weight_threshold = 35
    diastolic_bp_threshold = 35
    systolic_bp_threshold = 65
    heart_rate_threshold = 45

    # Correct the outliers
    train.loc[train['Physical-BMI'] <= bmi_threshold, 'Physical-BMI'] = bmi_threshold
    train.loc[train['Physical-Weight'] <= weight_threshold, 'Physical-Weight'] = weight_threshold
    train.loc[train['Physical-Diastolic_BP'] < diastolic_bp_threshold, 'Physical-Diastolic_BP'] = diastolic_bp_threshold
    train.loc[train['Physical-Systolic_BP'] < systolic_bp_threshold, 'Physical-Systolic_BP'] = systolic_bp_threshold
    train.loc[train['Physical-HeartRate'] < heart_rate_threshold, 'Physical-HeartRate'] = heart_rate_threshold
    swap_condition = train['Physical-Diastolic_BP'] > train['Physical-Systolic_BP']
    train.loc[swap_condition, ['Physical-Diastolic_BP', 'Physical-Systolic_BP']] = train.loc[swap_condition, ['Physical-Systolic_BP', 'Physical-Diastolic_BP']].values
    
    return train

def handle_outliers(train): 
    '''
    The following function handle the outliers, both the statistical domain knowledge. 
    The function receives from the user 
    train - the train dataframe set
    '''
    
    train_capper = cap_outliers(train,train.select_dtypes(include='number').columns)
    display(train_capper.describe())
    
    return train_capper


def high_correlation_pairs(train, threshold=0.95):
    # Calculate the correlation matrix
    corr_matrix = train.select_dtypes(include='number').corr("pearson")
    
    # Select pairs of features with correlations above the threshold
    high_corr_pairs = (
        corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))  # Upper triangle without diagonal
        .stack()  # Convert to Series
        .reset_index()
    )
    
    # Rename columns for readability
    high_corr_pairs.columns = ['Feature 1', 'Feature 2', 'Correlation']
    
    # Filter by the correlation threshold (both positive and negative)
    high_corr_pairs = high_corr_pairs[high_corr_pairs['Correlation'].abs() > threshold]
    
    # Display the high correlation pairs
    print("Highly correlated feature pairs (|correlation| > {}):".format(threshold))
    print(high_corr_pairs.to_string(index=False))
    
    return high_corr_pairs


# Find features with less then threshold correlation
def low_correlated_features(train, target_column, threshold=0.1):

    corr_matrix = train.corr()
    target_correlations = corr_matrix[target_column].abs()
    low_correlated_features = target_correlations[target_correlations < threshold].index
    
    return low_correlated_features

In [ ]:
def feature_engineering(df,tst):
    '''
        This function is used to clean the data and make sure it is ready for modeling
        input:
            train: the test dataframe
            test: the train dataframe           
    '''
    
    train = df.copy()
    test= tst.copy()
    ### Drop unique columns for train dataframe (later add the target series as it was dropped)
    cols_diff = feature_difference(train, test) 
    train = train.drop(list(cols_diff),axis=1)
    train = concat_df_a_target(train, Target_series)
    
    display(train)
    ### Feature creation 
    #feature_creation(train,test)
    
    ### Handle missing values
    train = train.dropna(subset='sii').reset_index().drop('index',axis=1)
    
    ### Drop high correlation pairs
    high_corr_pairs = high_correlation_pairs(train)
    # take the second feature from each pair and drop them from the dataframe
    features_to_remove = high_corr_pairs['Feature 2'].tolist()
    train = train.drop(features_to_remove, axis=1)
    test = test.drop(features_to_remove, axis=1)

    ### Drop features with low correlation to target
    low_corr_cols = low_correlated_features(train.select_dtypes(include='number'),'sii')
    train = train.drop(low_corr_cols,axis=1)
    test = test.drop(low_corr_cols,axis=1)

    ### Deal with outliers
    train = handle_outliers(train)

    ### Drop categorical columns 
    cat_cols = train.select_dtypes(exclude='number').columns
    train = train.drop(cat_cols,axis=1)
    test = test.drop(cat_cols,axis=1)
    
    return train, test 

In [ ]:
train_processed,test_processed = feature_engineering(train,test)

,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,...,PAQ_A-Season,PAQ_A-PAQ_A_Total,PAQ_C-Season,PAQ_C-PAQ_C_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii
0,Fall,5,0,Winter,51.0,Fall,16.877316,46.0,50.8,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fall,3.0,2.0
1,Summer,9,0,NaN,NaN,Fall,14.035590,48.0,46.0,22.0,...,NaN,NaN,Fall,2.340,Fall,46.0,64.0,Summer,0.0,0.0
2,Summer,10,1,Fall,71.0,Fall,16.648696,56.5,75.6,NaN,...,NaN,NaN,Summer,2.170,Fall,38.0,54.0,Summer,2.0,0.0
3,Winter,9,0,Fall,71.0,Summer,18.292347,56.0,81.6,NaN,...,NaN,NaN,Winter,2.451,Summer,31.0,45.0,Winter,0.0,1.0
4,Spring,18,1,Summer,NaN,NaN,NaN,NaN,NaN,NaN,...,Summer,1.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3955,Fall,13,0,Spring,60.0,Fall,16.362460,59.5,82.4,NaN,...,NaN,NaN,Winter,3.260,Winter,35.0,50.0,Fall,1.0,1.0
3956,Winter,10,0,NaN,NaN,Spring,18.764678,53.5,76.4,27.0,...,NaN,NaN,Winter,2.340,NaN,NaN,NaN,Winter,0.0,NaN
3957,Fall,11,0,Spring,68.0,Winter,21.441500,60.0,109.8,NaN,...,NaN,NaN,Winter,2.729,Winter,56.0,77.0,Fall,0.0,1.0
3958,Spring,13,0,Spring,70.0,Winter,12.235895,70.7,87.0,NaN,...,NaN,NaN,Spring,3.300,Spring,33.0,47.0,Spring,1.0,0.0


Highly correlated feature pairs (|correlation| > 0.95):
        Feature 1       Feature 2  Correlation
     Physical-BMI     BIA-BIA_BMI     0.964411
      BIA-BIA_BMC     BIA-BIA_BMR     0.990015
      BIA-BIA_BMC     BIA-BIA_DEE     0.979736
      BIA-BIA_BMC     BIA-BIA_ECW     0.989881
      BIA-BIA_BMC     BIA-BIA_FFM     0.990015
      BIA-BIA_BMC     BIA-BIA_Fat    -0.992377
      BIA-BIA_BMC     BIA-BIA_ICW     0.979808
      BIA-BIA_BMC     BIA-BIA_LDM     0.994194
      BIA-BIA_BMC     BIA-BIA_LST     0.965831
      BIA-BIA_BMC     BIA-BIA_SMM     0.971972
      BIA-BIA_BMC     BIA-BIA_TBW     0.986766
      BIA-BIA_BMR     BIA-BIA_DEE     0.993667
      BIA-BIA_BMR     BIA-BIA_ECW     0.999182
      BIA-BIA_BMR     BIA-BIA_FFM     1.000000
      BIA-BIA_BMR     BIA-BIA_Fat    -0.979725
      BIA-BIA_BMR     BIA-BIA_ICW     0.997328
      BIA-BIA_BMR     BIA-BIA_LDM     0.998821
      BIA-BIA_BMR     BIA-BIA_LST     0.992721
      BIA-BIA_BMR     BIA-BIA_SMM     0.992483
    

,Basic_Demos-Age,Basic_Demos-Sex,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Systolic_BP,FGC-FGC_CU,FGC-FGC_GSND,FGC-FGC_GSD,FGC-FGC_PU,FGC-FGC_SRL_Zone,FGC-FGC_TL,BIA-BIA_FFMI,BIA-BIA_Frame_num,SDS-SDS_Total_Raw,PreInt_EduHx-computerinternet_hoursday,sii
count,2736.000000,2736.000000,2527.000000,2530.000000,2572.000000,483.000000,2478.000000,1919.000000,872.000000,871.000000,1909.000000,1877.000000,1919.000000,1813.000000,1813.000000,2527.000000,2654.000000,2736.000000
mean,10.211257,0.364401,18.951877,55.897051,86.981038,26.469979,116.581517,11.235018,21.676293,22.730941,5.484285,0.636654,9.118525,14.578091,1.733591,40.804907,1.015072,0.580409
std,3.355036,0.481350,4.232603,7.394938,40.609535,4.758551,15.234210,9.644597,8.771454,9.110844,6.417924,0.481091,2.950591,1.606253,0.671201,9.701152,1.080861,0.771122
min,5.000000,0.000000,7.675152,36.000000,0.000000,19.000000,80.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.395100,1.000000,17.000000,0.000000,0.000000
25%,8.000000,0.000000,15.773447,50.050000,57.200000,23.000000,107.000000,4.000000,15.100000,16.100000,0.000000,0.000000,7.000000,13.381000,1.000000,33.000000,0.000000,0.000000
50%,10.000000,0.000000,17.819010,55.000000,75.800000,26.000000,114.000000,10.000000,19.400000,20.800000,3.000000,1.000000,9.000000,14.062900,2.000000,39.000000,1.000000,0.000000
75%,12.000000,1.000000,21.172311,61.750000,111.450000,29.000000,125.000000,16.000000,26.325000,27.900000,9.000000,1.000000,12.000000,15.371600,2.000000,46.000000,2.000000,1.000000
max,18.000000,1.000000,29.270606,78.500000,192.825000,38.000000,152.000000,34.000000,43.162500,45.600000,22.500000,1.000000,19.500000,18.357500,3.000000,65.500000,3.000000,3.000000


<h2>Combine sets</h2>
Combine the categrocial features dataset with the numerical features dataset
</br>
For the categorical features
I performed correlation in another file and chose the features with the highest correlation 

In [ ]:
train_dummies = pd.get_dummies(train)
train_dummies["sii"]

0       2.0
1       0.0
2       0.0
3       1.0
4       NaN
       ... 
3955    1.0
3956    NaN
3957    1.0
3958    0.0
3959    NaN
Name: sii, Length: 3960, dtype: float64

In [ ]:
### Define the best categorical feature based on correlation to the target feature 
best_categorical_features = ["SDS-Season_Spring","SDS-Season_Winter","SDS-Season_Fall","CGAS-Season_Spring","SDS-Season_Summer",
                             "CGAS-Season_Winter","Fitness_Endurance-Season_Spring","CGAS-Season_Fall","BIA-Season_Fall"]

# Convert dataframes to dummies 
train_dummies = pd.get_dummies(train)
test_dummies = pd.get_dummies(test)

# keep only the column that were defined in the beggining 
train_dummies = train_dummies.dropna(subset="sii").reset_index().drop('index',axis=1)
train_cat_best = train_dummies[best_categorical_features]
test_cat_best = test_dummies[best_categorical_features]

In [ ]:
# Concat between the categorical dataframe and the numerical dataframe 
train = pd.concat([train_processed, train_cat_best], axis=1, ignore_index=False)
test = pd.concat([test_processed, test_cat_best], axis=1, ignore_index=False)

<h1> Evaluation </h1>
Evaluate the model using the QWK metric 

In [ ]:
def quadratic_weighted_kappa(y_true, y_pred):
    return cohen_kappa_score(y_true, y_pred, weights='quadratic')

def threshold_Rounder(oof_non_rounded, thresholds):
    return np.where(oof_non_rounded < thresholds[0], 0,
                    np.where(oof_non_rounded < thresholds[1], 1,
                             np.where(oof_non_rounded < thresholds[2], 2, 3)))

def evaluate_predictions(thresholds, y_true, oof_non_rounded):
    rounded_p = threshold_Rounder(oof_non_rounded, thresholds)
    return -quadratic_weighted_kappa(y_true, rounded_p)


def TrainML(model_class, test_data):
    X = train.drop(['sii'], axis=1)
    y = train['sii']

    SKF = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    
    train_S = []
    test_S = []
    
    oof_non_rounded = np.zeros(len(y), dtype=float) 
    oof_rounded = np.zeros(len(y), dtype=int) 
    test_preds = np.zeros((len(test_data), n_splits))

    for fold, (train_idx, test_idx) in enumerate(tqdm(SKF.split(X, y), desc="Training Folds", total=n_splits)):
        X_train, X_val = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[test_idx]

        model = clone(model_class)
        model.fit(X_train, y_train)

        y_train_pred = model.predict(X_train)
        y_val_pred = model.predict(X_val)

        oof_non_rounded[test_idx] = y_val_pred
        y_val_pred_rounded = y_val_pred.round(0).astype(int)
        oof_rounded[test_idx] = y_val_pred_rounded

        train_kappa = quadratic_weighted_kappa(y_train, y_train_pred.round(0).astype(int))
        val_kappa = quadratic_weighted_kappa(y_val, y_val_pred_rounded)

        train_S.append(train_kappa)
        test_S.append(val_kappa)
        
        test_preds[:, fold] = model.predict(test_data)
        
        print(f"Fold {fold+1} - Train QWK: {train_kappa:.4f}, Validation QWK: {val_kappa:.4f}")
        clear_output(wait=True)

    print(f"Mean Train QWK --> {np.mean(train_S):.4f}")
    print(f"Mean Validation QWK ---> {np.mean(test_S):.4f}")

    KappaOPtimizer = minimize(evaluate_predictions,
                              x0=[0.5, 1.5, 2.5], args=(y, oof_non_rounded), 
                              method='Nelder-Mead')
    assert KappaOPtimizer.success, "Optimization did not converge."
    
    oof_tuned = threshold_Rounder(oof_non_rounded, KappaOPtimizer.x)
    tKappa = quadratic_weighted_kappa(y, oof_tuned)

    print(f"----> || Optimized QWK SCORE :: {Fore.CYAN}{Style.BRIGHT} {tKappa:.3f}{Style.RESET_ALL}")

    tpm = test_preds.mean(axis=1)
    tpTuned = threshold_Rounder(tpm, KappaOPtimizer.x)
    
    submission = pd.DataFrame({
        'id': sample['id'],
        'sii': tpTuned
    })

    return submission

In [ ]:
Light = LGBMRegressor(verbose = -1)
XGB_Model = XGBRegressor()
CatBoost_Model = CatBoostRegressor(verbose = False)


voting_model = VotingRegressor(estimators=[
    ('lightgbm', Light),
    ('xgboost', XGB_Model),
    ('catboost', CatBoost_Model)
])

Submission = TrainML(voting_model, test)

Submission

Training Folds: 100%|██████████| 5/5 [01:16<00:00, 15.24s/it]

Mean Train QWK --> 0.9449
Mean Validation QWK ---> 0.3897


----> || Optimized QWK SCORE ::  0.431


,id,sii
0,00008ff9,2
1,000fd460,0
2,00105258,0
3,00115b9f,0
4,0016bb22,1
5,001f3379,1
6,0038ba98,0
7,0068a485,0
8,0069fbed,1
9,0083e397,1


In [ ]:
Submission.to_csv("submission.csv",index=False)